In [8]:
import os
import shutil
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader, ConcatDataset, random_split
from torchvision import transforms,utils,models
import matplotlib.pyplot as plt

# 1.MyData数据模块
class MyData(Dataset):

    """
    root_dir：包含训练集train、测试集test的总目录路径
    mode：选哪一个数据集？train或test
    label：选猫猫(1)或狗狗(0)
    flag：1表示需要划分数据集（train, test）；0表示不需要划分数据集，注意仅用一次!
    """

    def __init__(self, root_dir, mode, label, flag=0): # 初始化数据集及成员root_dir、mode、label
        self.root_dir = root_dir
        self.mode = mode
        self.label = label
        if flag==1:
            # 将Cats数据集中的猫猫图片移动到train/cats、test/cats中，比例分别为8:2
            # 创建文件夹
            if not os.path.exists(root_dir+'\\train\\cats'):
                os.makedirs(os.path.join(root_dir,'train\\cats'))
            if not os.path.exists(root_dir+'\\test\\cats'):
                os.makedirs(os.path.join(root_dir,'test\\cats'))

            # 分割train, test
            cats_path = os.listdir(os.path.join(root_dir,'Cats')) # 获取猫猫图片地址
            num_cats = len(cats_path)
            train_cats, test_cats = random_split(
                dataset = cats_path,
                lengths = [int(num_cats*0.8), num_cats-int(num_cats*0.8)])

            # 移动文件，并删除原Cats文件夹
            for i in list(train_cats):
                shutil.move(os.path.join(root_dir, 'Cats', i), os.path.join(root_dir,'train\\cats'))
            for i in list(test_cats):
                shutil.move(os.path.join(root_dir, 'Cats', i), os.path.join(root_dir,'test\\cats'))  
            os.rmdir(os.path.join(root_dir,'Cats'))

            # 将Dogs数据集中的狗狗图片移动到train/dogs、test/dogs中，比例分别为8:2
            # 创建文件夹
            if not os.path.exists(root_dir+'\\train\\dogs'):
                os.makedirs(os.path.join(root_dir,'train\\dogs'))
            if not os.path.exists(root_dir+'\\test\\dogs'):
                os.makedirs(os.path.join(root_dir,'test\\dogs'))

            # 分割train, test
            dogs_path = os.listdir(os.path.join(root_dir,'Dogs')) # 获取狗狗图片地址
            num_dogs = len(dogs_path)
            train_dogs, test_dogs = random_split(
                dataset = dogs_path,
                lengths = [int(num_dogs*0.8), num_dogs-int(num_dogs*0.8)])

            # 移动文件，并删除原Dogs文件夹
            for j in list(train_dogs):
                shutil.move(os.path.join(root_dir, 'Dogs', j), os.path.join(root_dir,'train\\dogs'))
            for j in list(test_dogs):
                shutil.move(os.path.join(root_dir, 'Dogs', j), os.path.join(root_dir,'test\\dogs'))  
            os.rmdir(os.path.join(root_dir,'Dogs'))

        self.img_path = os.listdir(os.path.join(root_dir, mode, label)) # 获取所有图片数据名称
        
    def __len__(self): # 获取图片数据size
        return len(self.img_path)
    
    def __getitem__(self, index): # 获取指定图片数据及名称
        img_item_name = self.img_path[index]
        img_item_path = os.path.join(self.root_dir, self.mode, self.label, img_item_name)
        img = Image.open(img_item_path).convert('RGB')
        transform = transforms.Compose([  # 图像变换
                    transforms.Resize((224, 224)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.2,0.2,0.2], std=[0.5,0.5,0.5])
                ])
        img = transform(img)
        if img_item_name[:3]=='cat':
            img_item_name = 1
        else:
            img_item_name = 0
        return img, img_item_name

In [9]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('./logs')

# 2.tensorboard数据预览
root_dir = '.\\data'
mode = 'train'
label = 'cats'
train_cats_dataset = MyData(root_dir, 'train', 'cats', flag=0)

loader = DataLoader(train_cats_dataset, batch_size=30, shuffle=True , num_workers=0, drop_last=False) # 将全部图片随机加载为30个一组的图片数据（不采取多线程加载，不丢弃剩余数据）
step = 0
for data in loader:
    imgs,labels = data
    writer.add_images('images',imgs,step)
    step = step+1
writer.close()

In [10]:
# 3.加载数据
# 训练数据集
train_cats_dataset = MyData(root_dir, 'train', 'cats', flag=0)
train_dogs_dataset = MyData(root_dir, 'train', 'dogs', flag=0)
# train_dataset = ConcatDataset([train_cats_dataset, train_dogs_dataset]) # 合并猫狗数据集
train_dataset = train_cats_dataset+train_dogs_dataset

# 测试数据集
test_cats_dataset = MyData(root_dir, 'test', 'cats', flag=0)
test_dogs_dataset = MyData(root_dir, 'test', 'dogs', flag=0)
test_dataset = ConcatDataset([test_cats_dataset, test_dogs_dataset])

# 加载数据
train_loader = DataLoader(dataset=train_dataset, batch_size=10, shuffle=True, num_workers=0)
test_loader = DataLoader(dataset=test_dataset, batch_size=10, shuffle=True, num_workers=0)

In [11]:
import torch
import torch.nn as nn # Neural networks模块，Pytorch最核心的模块，用于自主构建神经网络框架
from torch.nn import Sequential # 用于集成神经网路各组件，作用类似于transforms中的compose

# 4.构建Alexnet卷积神经网络框架
class MyNet(nn.Module): # 继承
    def __init__(self):
        super(MyNet,self).__init__()
        """
        基本概念：
        conv2d:图像卷积，局部特征提取
            in_channels为输入通道数（特殊的，首次卷积操作的in_channels指图片颜色通道数，通常为3（RGB三通道）或1（灰度图）
            out_channels为输出通道数，即卷积核的个数，除首次卷积操作外，此后上一层卷积out_channels = 下一层卷积in_channels
            kernel_size为卷积核高宽尺寸（卷积核深度=in_channels）
            stride为卷积核扫描步幅
            padding为边缘填充尺寸（用空白填充）

        ReLU:非线性激活（注意区分激活层与全连接层）

        MaxPool2d：局部特征再提取，减少参数，增强鲁棒性
            kernel_size为池化核尺寸（注意池化核只有2个维度，池化操作在每个通道上同步进行）
            stride为池化核扫描步幅
            padding为边缘填充尺寸

        Flatten：高阶张量展平
            start_dim为开始展平的对应阶
            end_dim为结束展平的对应阶
            默认start_dim=1，end_di-1.即除第一阶外，其余阶全部展平

        Linear：全连接线性映射y=Ax+b，模仿生物神经元连接
            in_features为输入数据所含元素（特征）个数，如图像数据中元素个数 = 通道数*高*宽
            out_features为输出数据所含元素（特征）个数
            全连接层首先将输入的x展平为一维列向量，然后用二维矩阵A对其作左乘积运算，输出列向量y长度为矩阵行数

        Dropout：对输入数据的每个元素（特征）依概率置零，表现为后续全连接部分节点失活，作用为防止过拟合
            p为概率阈值，若某特征所取随机数（0-1）小于p则置零
        """
    
        """
        卷积层一：
        Conv2d：输入224x224x3 卷积核 96个11x11x3，步幅为4，不扩充边缘，
                因此输出为高((224-11+0x2)//4)+1 = 54  宽((224-11+0x2)//4)+1 = 54  通道96
        ReLU：输入输出不变
        MaxPool2d：输入54x54x96，池化核 3x3，步幅为2，不扩充边缘，
                因此输出为高((54-3+0x2)//2)+1 = 26  宽((54-3+0x2)//2)+1 = 26  通道96
        """
        self.conv1 = Sequential(
            nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11, stride=4, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
            )

        """
        卷积层二：
        Conv2d：输入26x26x96 卷积核 256个5x5x96，步幅为1，扩充边缘2，
                因此输出为高((26-5+2x2)//1)+1 = 26  宽((26-5+2x2)//1)+1 = 26  通道256
        ReLU：输入输出不变
        MaxPool2d：输入26x26x256，池化核 3x3，步幅为2，不扩充边缘，
                因此输出为高((26-3+0x2)//2)+1 = 12  宽((26-3+0x2)//2)+1 = 12  通道256
        """
        self.conv2 = Sequential(
            nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
            )
        
        """
        卷积层三：
        Conv2d：输入12x12x256 卷积核 384个3x3x256，步幅为1，扩充边缘1，
                因此输出为高((12-3+1x2)//1)+1 = 12  宽((12-3+1x2)//1)+1 = 12  通道384
        ReLU：输入输出不变
        """
        self.conv3 = Sequential(
            nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride=1, padding=1),
            nn.ReLU()
            )
        
        """
        卷积层四：
        Conv2d：输入12x12x384 卷积核 384个3x3x384，步幅为1，扩充边缘1，
                因此输出为高((12-3+1x2)//1)+1 = 12  宽((12-3+1x2)//1)+1 = 12  通道384
        ReLU：输入输出不变
        """
        self.conv4 = Sequential(
            nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1),
            nn.ReLU()
            )

        """
        卷积层五：
        Conv2d：输入12x12x384 卷积核 256个3x3x384，步幅为1，扩充边缘1，
                因此输出为高((12-3+1x2)//1)+1 = 12  宽((12-3+1x2)//1)+1 = 12  通道256
        ReLU：输入输出不变
        MaxPool2d：输入12x12x256，池化核 3x3，步幅为2，不扩充边缘，
                因此输出为高((12-3+0x2)//2)+1 = 5  宽((12-3+0x2)//2)+1 = 5  通道256
        """
        self.conv5 = Sequential(
            nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
            )
        
        """
        全连接层1：
        nn.Flatten：输入5x5x256 经展平后为4096
        nn.Linear：输入4096并全连接（此处等价于用4096个5x5x256的卷积核进行扫描）
        ReLU：输入输出不变
        Dropout：输入输出不变
        """
        self.dense1 = Sequential(
            nn.Flatten(),
            nn.Linear(in_features=256*5*5, out_features=4096),
            nn.ReLU(),
            nn.Dropout(p=0.1)
            )
        
        """
        全连接层2：
        nn.Linear：输入输出均为4096
        ReLU：输入输出不变
        Dropout：输入输出不变
        """
        self.dense2 = Sequential(
            nn.Linear(in_features=4096, out_features=4096),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            )
        
        """
        全连接层3：
        nn.Linear：输入4096，输出2
        """
        self.dense3 = nn.Linear(in_features=4096, out_features=2)

    def forward(self, x):  # 正向传播
        x1 = self.conv1(x)
        x2 = self.conv2(x1)
        x3 = self.conv3(x2)
        x4 = self.conv4(x3)
        x5 = self.conv5(x4)
        x6 = self.dense1(x5)
        x7 = self.dense2(x6)
        x8 = self.dense3(x7)
        return x8


# 网络实例化及CUDA调用（必备的一句话）
device = "cuda" if torch.cuda.is_available() else "cpu" # 定义训练设备gpu or cpu
print("Using device:"+str(device))
Alexnet = MyNet().to(device) # 把模型加载进设备中

Using device:cuda


In [12]:
# 加载预训练模型resnet
# import torchvision.models as model
# Alexnet = model.resnet.resnet18(pretrained=False).to(device)

In [13]:
# 5.损失函数及优化器设置
"""
常用损失函数如下：
回归任务：nn.MSELoss（均方误差损失）
分类任务：nn.CrossEntropyLoss（交叉熵损失）

常用优化器如下：
SGD（随机梯度下降)：在简单的模型中效果较好，但在复杂的模型中可能会发生振荡现象。
Momentum:相对于SGD，Momentum在处理高维空间的梯度下降问题时更有效。
RMSprop:具有自适应学习率的特性，在处理高方差梯度的情况下比较有效。
Adam：在许多实际应用中表现很好，因为它同时兼顾了 Momentum和RMSprop的特性。（优先考虑）
Adagrad:适用于 稀疏数据的情况，因为它会自适应调整每个参数的学习率。
Adadelta：具有自适应学习率的优秀特性，可以减少学习率对结果的影响。
"""

# 损失函数 
loss_F = nn.CrossEntropyLoss() 

# 优化器
optimizer=torch.optim.SGD(params=Alexnet.parameters(), lr=0.05) # params为待优化参数，lr为学习率


In [ ]:
# 6.模型训练，打印和保存
"""
model.train（此处为cnn.train）：训练模式下，启用Batch Normalization层和Dropout层
model.eval（此处为cnn.eval）：验证或测试模式下，Batch Normalization层的均值方差沿用训练过程的结果，不启用Dropout
with torch.no_grad()：停止autograd模块的工作，以起到加速和节省GPU算力的作用

torch.save(xx)：此处用于保存Alexnet训练后的网络参数（字典形式）
model.load_state_dict(torch.load(xx))：用于加载保存的参数
"""

for epoch in range(1,3):
    print("Epoch [{}/{}]".format(epoch, 50))

    # train 训练环节
    train_loss = 0; train_accuracy = 0
    Alexnet.train()
    for data in train_loader: # 从loader中加载数据
        imgs_train, labels_train = data # 获取特征及标签
        imgs_train, labels_train = imgs_train.to(device), labels_train.to(device) # 把数据加载进设备中，设备为cpu的话无所谓
        outputs_train = Alexnet(imgs_train) # 网络前向传播计算，输出预测值（注意输入必须为一个batch）

        optimizer.zero_grad() # 梯度置零，避免上一步的梯度累积
        loss = loss_F(outputs_train, labels_train) # 根据预测值和真实标签计算损失
        loss.backward() # 网络反向传播计算，根据损失更新参数
        optimizer.step()

        train_loss = train_loss+loss.item() # 累计每个batch的训练损失
        train_accuracy = train_accuracy+torch.sum(outputs_train.argmax(1) == labels_train) # 累计每个batch的训练准确数
    train_loss = train_loss/len(train_dataset) # 计算训练环节损失
    train_accuracy = train_accuracy/len(train_dataset) # 计算训练环节准确率

    # test 测试环节（与训练相比，少了参数更新这一步）
    test_loss = 0; test_accuracy = 0
    Alexnet.eval()
    with torch.no_grad():
        for data in test_loader:
            imgs_test, labels_test = data
            imgs_test, labels_test = imgs_test.to(device), labels_test.to(device)
            outputs_test = Alexnet(imgs_test)

            test_loss = test_loss+loss_F(outputs_test, labels_test).item() 
            test_accuracy = test_accuracy+torch.sum(outputs_test.argmax(1) == labels_test) 
        test_loss = test_loss/len(test_dataset)
        test_accuracy = test_accuracy/len(test_dataset)

    # save 模型保存
    torch.save(Alexnet.state_dict(), './model_save/Alexnet_state_dict.pth')

    # write+print 模型训练测试结果打印
    writer.add_scalars("train",{"loss":train_loss, "accuracy":train_accuracy},epoch) # 绘制训练效果图
    writer.add_scalars("test",{"loss":test_loss, "accuracy":test_accuracy},epoch) # 绘制测试效果图
    writer.add_scalars("accuracy",{"accuracy_train":train_accuracy, "accuracy_test":test_accuracy},epoch)
    print("train_loss: {:.4f}  train_accuracy: {:.4f}  test_loss: {:.4f}  test_accuracy: {:.4f}".format(
          train_loss,train_accuracy, test_loss, test_accuracy))
writer.close()

In [ ]:
# 7.模型预测（以测试数据集为例）
Alexnet = MyNet()
Alexnet.load_state_dict(torch.load('./model_save/Alexnet_state_dict.pth')) # 加载训练好的网络参数
Alexnet.eval()

for img,label in test_dataset:
    result= lambda x:"猫" if x==1 else "狗" # lambda匿名函数
    output = Alexnet(img.reshape(1,3,224,224)).argmax(1)
    print("真实图片为:{}  预测图片为:{}".format(result(label),result(output)))

In [1]:
import netron
netron.start('./model_save/Alexnet_state_dict.pth')

Serving './model_save/Alexnet_state_dict.pth' at http://localhost:8080


('localhost', 8080)